In [1]:
library(tidyverse)
library(repr)
library(dplyr)
library(tidymodels)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

In [2]:
G20_economic_data <- read_csv("https://raw.githubusercontent.com/mparhar1/DSCI-100-Group-48-Project/main/WEOApri23G20.csv")
colnames(G20_economic_data) <- make.names(colnames(G20_economic_data), unique = TRUE)

Rows: 836 Columns: 59
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (46): ISO, WEO Subject Code, Country, Subject Descriptor, Subject Notes,...
dbl  (2): WEO Country Code, Estimates Start After

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [4]:
tidy_G20_economic_data <- G20_economic_data |>
    select(Country,
           Subject.Descriptor,
           Units,
           X2010:X2023
          ) |>
    filter(Subject.Descriptor == "Gross domestic product per capita, current prices" & Units == "U.S. dollars" |
           Subject.Descriptor == "Volume of imports of goods and services" |
           Subject.Descriptor == "Volume of exports of goods and services" | 
           Subject.Descriptor == "Inflation, average consumer prices" & Units == "Percent change" |
           Subject.Descriptor == "Unemployment rate" 
          ) |>
    filter(Country != "Argentina" & Country != "Saudi Arabia") |>
    mutate(X2014 = as.numeric(gsub(",","", X2014)),
           X2015 = as.numeric(gsub(",","", X2015)),
           X2016 = as.numeric(gsub(",","", X2016)),
           X2022 = as.numeric(gsub(",","", X2022)),
           X2023 = as.numeric(gsub(",","", X2023)),
          ) |>
    pivot_longer(cols = X2010:X2023,
                 names_to = "Year",
                 values_to = "Metric"
                ) |>
    group_by(Country, Year, Subject.Descriptor) |>
    summarise(Metric = mean(Metric, na.rm = TRUE)) |>
    pivot_wider(names_from = Subject.Descriptor,
                values_from = Metric
               )

colnames(tidy_G20_economic_data) <- make.names(colnames(tidy_G20_economic_data), unique = TRUE)



`summarise()` has grouped output by 'Country', 'Year'. You can override using
the `.groups` argument.


In [6]:
set.seed(100)
gdp_split <- initial_split(tidy_G20_economic_data, prop = 0.70, strata = Gross.domestic.product.per.capita..current.prices)
gdp_training <- training(gdp_split)
gdp_testing <- testing(gdp_split)


In [9]:

lm_spec <- linear_reg() |>
        set_engine("lm") |>
        set_mode("regression")

lm_recipe <- recipe(Gross.domestic.product.per.capita..current.prices ~ Inflation..average.consumer.prices + Unemployment.rate + Volume.of.exports.of.goods.and.services + Volume.of.imports.of.goods.and.services, data = gdp_training)

lm_fit <- workflow() |>
        add_recipe(lm_recipe) |>
        add_model(lm_spec) |>
        fit(data = gdp_training)

lm_fit

══ Workflow [trained] ══════════════════════════════════════════════════════════
Preprocessor: Recipe
Model: linear_reg()

── Preprocessor ────────────────────────────────────────────────────────────────
0 Recipe Steps

── Model ───────────────────────────────────────────────────────────────────────

Call:
stats::lm(formula = ..y ~ ., data = data)

Coefficients:
                            (Intercept)  
                                38632.1  
     Inflation..average.consumer.prices  
                                 -581.3  
                      Unemployment.rate  
                                 -836.3  
Volume.of.exports.of.goods.and.services  
                                 -284.9  
Volume.of.imports.of.goods.and.services  
                                  169.0  


In [12]:
lm_test_results <- lm_fit |>
                predict(gdp_testing) |>
                bind_cols(gdp_testing) |>
                metrics(truth = Gross.domestic.product.per.capita..current.prices, estimate = .pred)

lm_rmspe <- lm_test_results |>
            filter(.metric == "rmse") |>
            select(.estimate) |>
            pull()
lm_rmspe

[1] 19012.21

In [20]:
australia <- filter(tidy_G20_economic_data, Country == "Australia")
australia

Country,Year,Gross.domestic.product.per.capita..current.prices,Inflation..average.consumer.prices,Unemployment.rate,Volume.of.exports.of.goods.and.services,Volume.of.imports.of.goods.and.services
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Australia,X2010,56579.03,2.863,5.208,5.709,16.142
Australia,X2011,67270.36,3.356,5.083,-0.144,10.557
Australia,X2012,68450.31,1.686,5.233,5.420,6.183
Australia,X2013,65200.37,2.450,5.658,5.890,-2.284
Australia,X2014,61615.05,2.513,6.058,7.010,-1.410
Australia,X2015,51412.25,1.485,6.050,6.319,2.265
Australia,X2016,51813.64,1.277,5.708,6.673,0.131
Australia,X2017,55797.38,1.995,5.583,3.432,7.775
Australia,X2018,56341.94,1.933,5.292,5.112,4.275


In [21]:
set.seed(100)
aus_split <- initial_split(australia, prop = 0.70, strata = Gross.domestic.product.per.capita..current.prices)
aus_training <- training(aus_split)
aus_testing <- testing(aus_split)


Warning message:
“The number of observations in each quantile is below the recommended threshold of 20.
• Stratification will use 0 breaks instead.”
Warning message:
“Too little data to stratify.
• Resampling will be unstratified.”


In [22]:
lm_aus <- linear_reg() |>
        set_engine("lm") |>
        set_mode("regression")

lm_recipe_aus <- recipe(Gross.domestic.product.per.capita..current.prices ~ Inflation..average.consumer.prices + Unemployment.rate + Volume.of.exports.of.goods.and.services + Volume.of.imports.of.goods.and.services, data = aus_training)

lm_fit_aus <- workflow() |>
        add_recipe(lm_recipe) |>
        add_model(lm_spec) |>
        fit(data = aus_training)

lm_fit_aus

══ Workflow [trained] ══════════════════════════════════════════════════════════
Preprocessor: Recipe
Model: linear_reg()

── Preprocessor ────────────────────────────────────────────────────────────────
0 Recipe Steps

── Model ───────────────────────────────────────────────────────────────────────

Call:
stats::lm(formula = ..y ~ ., data = data)

Coefficients:
                            (Intercept)  
                               136702.1  
     Inflation..average.consumer.prices  
                                -4594.7  
                      Unemployment.rate  
                               -13520.2  
Volume.of.exports.of.goods.and.services  
                                 -161.7  
Volume.of.imports.of.goods.and.services  
                                  980.9  


In [23]:
lm_test_results_aus <- lm_fit_aus |>
                predict(aus_testing) |>
                bind_cols(aus_testing) |>
                metrics(truth = Gross.domestic.product.per.capita..current.prices, estimate = .pred)

lm_rmspe_aus <- lm_test_results_aus |>
            filter(.metric == "rmse") |>
            select(.estimate) |>
            pull()
lm_rmspe_aus

[1] 16318.89

In [ ]:
canada <- filter(tidy_G20_economic_data, Country == "Australia")
australia